<a href="https://colab.research.google.com/github/raja-jamwal/blog-building-agentic-architectures/blob/main/langchain/Part_7_Evaluation%2C_Prioritization%2C_and_Exploration.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Building Agents in Python and n8n in 2026
Companion to https://rajajamwal.substack.com/p/building-agents-in-python-and-n8n

Subscribe to my blog, https://rajajamwal.substack.com

## Evaluation, Prioritization, and Exploration

This is the **Final Chapter**.

We have built agents that can chain, route, reflect, collaborate, and use tools. We have secured them with guardrails and optimized them for cost.

Now, we face the final challenge: **How do we know if it's working?** And how do we make the agent work on the *right* things?

We will cover:
1.  **Evaluation (LLM-as-a-Judge):** Building an automated pipeline where one AI grades another AI's work.
2.  **Prioritization:** Teaching an agent to rank tasks based on urgency and impact.
3.  **Exploration:** Creating an agent that proactively seeks new information (the "Scientist" pattern).

### The Stack
*   **Python**
*   **LangChain**
*   **OpenAI** (GPT-4o-mini)

### The n8n Connection
*   **Evaluation** = Create a "Test Workflow". It runs your main agent, captures the output, and passes it to a second AI Agent node with a "Grading Rubric" prompt.
*   **Prioritization** = An AI Agent node that takes a JSON list of tasks and outputs a sorted JSON list. You then use the **Split In Batches** node to process the top priority items first.
*   **Exploration** = A loop where the AI generates search queries, fetches results (HTTP Request), and then uses those results to generate *new* queries (Loop Node).

In [1]:
# @title 1. Install Dependencies
!pip install -qU langchain langchain-openai langchain-core

import os
from getpass import getpass

# @title 2. Setup API Key
if "OPENAI_API_KEY" not in os.environ:
    os.environ["OPENAI_API_KEY"] = getpass("Enter your OpenAI API Key: ")

# Initialize the Model
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

print("✅ Environment Setup Complete.")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.7/84.7 kB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 489.1/489.1 kB 40.4 MB/s eta 0:00:00
Enter your OpenAI API Key: ··········
✅ Environment Setup Complete.


## Pattern 19: Evaluation (LLM-as-a-Judge)

**The Problem:** In traditional software, unit tests pass or fail (True/False). In AI, the output is text. How do you "test" if a summary is "good"?

**The Solution:** **LLM-as-a-Judge**. We use a strong model (the Judge) to evaluate the output of our worker model based on a specific rubric (Accuracy, Tone, Conciseness).

**The Scenario:** We have a "Junior Support Agent" answering questions. We will build a "Senior Auditor" to grade its answers.

In [2]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

# --- 1. The Subject (The Agent being tested) ---
student_prompt = ChatPromptTemplate.from_template(
    "Answer this customer question briefly: {question}"
)
student_chain = student_prompt | llm | StrOutputParser()

# --- 2. The Judge (The Evaluator) ---
judge_prompt = ChatPromptTemplate.from_template(
    """You are a Quality Assurance Auditor.
    Grade the following answer on a scale of 1 to 5 based on these criteria:
    1. Accuracy (Is it true?)
    2. Politeness (Is it nice?)
    3. Helpfulness (Does it solve the problem?)

    Question: {question}
    Agent Answer: {answer}

    Format:
    Score: [1-5]
    Reasoning: [One sentence explanation]
    """
)
judge_chain = judge_prompt | llm | StrOutputParser()

# --- Execution ---
questions = [
    "How do I reset my password?",
    "Your product is garbage and I hate it!"
]

print("--- 🧪 Starting Evaluation Loop ---")

for q in questions:
    print(f"\n📝 Question: {q}")

    # Step 1: Generate Answer
    answer = student_chain.invoke({"question": q})
    print(f"🤖 Agent Answer: {answer}")

    # Step 2: Evaluate
    evaluation = judge_chain.invoke({"question": q, "answer": answer})
    print(f"⚖️ Judge Verdict:\n{evaluation}")

--- 🧪 Starting Evaluation Loop ---

📝 Question: How do I reset my password?
🤖 Agent Answer: To reset your password, go to the login page and click on the "Forgot Password?" link. Follow the instructions sent to your email to create a new password.
⚖️ Judge Verdict:
Score: 5  
Reasoning: The answer is accurate, polite, and provides clear instructions that effectively solve the problem of resetting a password.

📝 Question: Your product is garbage and I hate it!
🤖 Agent Answer: I'm sorry to hear that you're not satisfied with our product. Your feedback is important to us, and we’d like to help resolve any issues you’re experiencing. Please let us know what specifically you dislike, and we’ll do our best to assist you.
⚖️ Judge Verdict:
Score: 5  
Reasoning: The agent's response is accurate in acknowledging the customer's dissatisfaction, polite in its tone, and helpful by inviting the customer to provide more details for resolution.


## Pattern 20: Prioritization (The Strategist)

**The Problem:** An agent receives 50 emails. If it processes them in order, it might spend all its time on spam and miss the urgent "Server Down" alert.

**The Solution:** **The Triage Step**. Before doing any work, the agent analyzes the queue and assigns a "Priority Score" to each item. It then sorts the list.

**The Scenario:** We have a list of support tickets. The agent must sort them so we handle the fires first.

In [3]:
import json

# --- The Input Data (Unsorted) ---
tickets = [
    {"id": 1, "subject": "I can't change my profile picture color"},
    {"id": 2, "subject": "URGENT: The payment gateway is down, losing money!"},
    {"id": 3, "subject": "Feature request: Dark mode"},
    {"id": 4, "subject": "Security Alert: Suspicious login detected"}
]

# --- The Prioritizer ---
# We ask the LLM to return a JSON list sorted by priority.
priority_prompt = ChatPromptTemplate.from_template(
    """You are a Support Manager.
    Analyze the following tickets.
    Assign a priority (High, Medium, Low) to each.
    Return a JSON list of the tickets SORTED by priority (High first).

    Tickets:
    {tickets}
    """
)

priority_chain = priority_prompt | llm | StrOutputParser()

# --- Execution ---
print("--- 📥 Incoming Tickets (Unsorted) ---")
for t in tickets:
    print(f"- {t['subject']}")

print("\n--- 🧠 Agent Prioritizing... ---")
sorted_response = priority_chain.invoke({"tickets": str(tickets)})

# Clean up the response to ensure it's valid JSON (sometimes LLMs add markdown)
cleaned_response = sorted_response.replace("```json", "").replace("```", "").strip()

try:
    sorted_tickets = json.loads(cleaned_response)
    print("\n--- 📤 Priority Queue (Sorted) ---")
    for t in sorted_tickets:
        # We expect the LLM to have added a 'priority' field
        priority = t.get('priority', 'Unknown')
        print(f"[{priority.upper()}] {t['subject']}")
except Exception as e:
    print(f"Error parsing JSON: {e}\nRaw Output: {sorted_response}")

--- 📥 Incoming Tickets (Unsorted) ---
- I can't change my profile picture color
- URGENT: The payment gateway is down, losing money!
- Feature request: Dark mode
- Security Alert: Suspicious login detected

--- 🧠 Agent Prioritizing... ---
Error parsing JSON: Expecting value: line 1 column 1 (char 0)
Raw Output: Here is the analysis of the tickets with assigned priorities:

1. Ticket 2: "URGENT: The payment gateway is down, losing money!" - This is a high priority as it directly affects revenue and requires immediate attention.
2. Ticket 4: "Security Alert: Suspicious login detected" - This is also a high priority due to the potential security risk involved.
3. Ticket 1: "I can't change my profile picture color" - This is a medium priority as it affects user experience but does not have immediate consequences.
4. Ticket 3: "Feature request: Dark mode" - This is a low priority as it is a request for a new feature rather than an issue that needs urgent resolution.

Here is the JSON list o

## Pattern 21: Exploration and Discovery (The Scientist)

**The Problem:** Most agents are "reactive" (you ask, they answer). But true intelligence involves "proactive" discovery asking questions to find information you didn't know you needed.

**The Solution:** **The Hypothesis Loop**. The agent looks at a topic, formulates a hypothesis, generates a search query to test it, and then refines its understanding.

**The Scenario:** We want to explore "The future of batteries." Instead of just summarizing Wikipedia, the agent will generate *novel questions* to investigate.

In [4]:
# --- The Explorer ---
explorer_prompt = ChatPromptTemplate.from_template(
    """You are a Scientific Researcher.
    Topic: {topic}
    Current Knowledge: {knowledge}

    Your goal is to uncover novel or contradictory information.
    Generate 3 search queries that would help you find "unknown unknowns" about this topic.
    Focus on emerging trends or problems.

    Return ONLY the 3 queries as a bulleted list.
    """
)

explorer_chain = explorer_prompt | llm | StrOutputParser()

# --- Execution ---
topic = "Solid State Batteries"
initial_knowledge = "They are safer and have higher energy density than Li-ion."

print(f"🔭 Exploring Topic: {topic}")
print(f"🧠 Starting Knowledge: {initial_knowledge}\n")

# Iteration 1: Generate Questions
queries = explorer_chain.invoke({
    "topic": topic,
    "knowledge": initial_knowledge
})

print("--- ❓ Generated Research Queries ---")
print(queries)

# Note: In a full system, you would now:
# 1. Feed these queries into a Search Tool (Google/Tavily).
# 2. Get the results.
# 3. Feed the results back into 'Current Knowledge'.
# 4. Repeat the loop to dig deeper.

🔭 Exploring Topic: Solid State Batteries
🧠 Starting Knowledge: They are safer and have higher energy density than Li-ion.

--- ❓ Generated Research Queries ---
- "emerging challenges in solid state battery manufacturing scalability"
- "recent breakthroughs in solid state battery materials and their implications"
- "environmental impact of solid state batteries compared to lithium-ion technology"


You now possess a toolkit of **21 Patterns** ranging from basic Prompt Chaining to advanced Evaluation and Exploration.

### What's Next?
1.  **Combine Them:** Real systems use 5-10 patterns at once. (e.g., RAG + Routing + Guardrails + Evaluation).
2.  **Move to n8n:** Take these Python concepts and visualize them in n8n workflows for rapid deployment.
3.  **Build:** The only way to master this is to build something. Go create an agent that solves a real problem for you.

Thank you for coding along!